In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [3]:
# Reload the data from the pickle file generated in Assignment 1
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
# Reformat the pickled data. Turn the image data itself into a 2D (flat) matrix and turn the 
# labels into separate 1-hot encoded vectors
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
# Define an accuracy function to calculate the accuracy of the subsequent networks
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [36]:
# Problem 1
# This code initializes a computational graph for this data in TensorFlow. Mostly copied from Assignment 2
# except for the addition of L2 regularization into the loss calculation

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  # We also add L2 regularization to the loss by adding the L2 norm of the loss along with a variable parameter
  # beta
  logits = tf.matmul(tf_train_dataset, weights) + biases
  beta = 0.1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
      (beta * tf.nn.l2_loss(weights)))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [37]:
# This code evaluates the above logistic classifier.
# Before addition of L2 regularization:
#    Final Validation accuracy = 75.3%
#    Final Test accuracy = 82.7%
# After addition of L2 regularization:
#    Final Validation accuracy = 80.4%
#    Final Test accuracy = 87.2%


num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 318.697784
Training accuracy: 7.8%
Validation accuracy: 11.0%
Loss at step 100: 1.003937
Training accuracy: 81.1%
Validation accuracy: 80.3%
Loss at step 200: 0.991441
Training accuracy: 81.3%
Validation accuracy: 80.4%
Loss at step 300: 0.990575
Training accuracy: 81.4%
Validation accuracy: 80.4%
Loss at step 400: 0.990321
Training accuracy: 81.5%
Validation accuracy: 80.4%
Loss at step 500: 0.990238
Training accuracy: 81.5%
Validation accuracy: 80.3%
Loss at step 600: 0.990209
Training accuracy: 81.4%
Validation accuracy: 80.4%
Loss at step 700: 0.990198
Training accuracy: 81.4%
Validation accuracy: 80.4%
Loss at step 800: 0.990194
Training accuracy: 81.4%
Validation accuracy: 80.4%
Test accuracy: 87.2%


In [38]:
# Problem 1 (continued)

# The below code creates a 2-layer neural network from the logistic classifier above. It then evaluates 
# the network on the training, validation and test sets.

# This code is copied from Assignment 2 aside from the L2 regularization added to the loss function.

batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  # The shape of weights_in must be the same as the number of columns of the training set times 
  # the number of hidden nodes (each node is a ReLU that will predict one specific thing)
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
  # The shape of weights_out must be the same as the number of columns of weights_in times 
  # the number of labels to finally be output
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation. This was made into a function because this series of operations is performed
  # multiple times in addition to trianing when calculating predictions for the validation and test 
  # sets
  def training_comps(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes layer_1 to be based on ReLU's
      out_layer = tf.matmul(in_layer, out_weights) + out_biases
      return out_layer
  # This beta is the hyperparameter for the L2 regularization
  beta = 0.005
  logits = training_comps(tf_train_dataset, weights_in, biases_in, weights_out, biases_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    (beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits) # Running the softmax on the logits completes the network
  # Run the training computations for the validation and test sets
  valid_prediction = tf.nn.softmax(training_comps(tf_valid_dataset, weights_in, biases_in, weights_out, biases_out))
  test_prediction = tf.nn.softmax(training_comps(tf_test_dataset, weights_in, biases_in, weights_out, biases_out))

# Network Evaluation

# Before addition of L2 regularization:
#    Final Validation accuracy = 81.5%
#    Final Test accuracy = 88.4%
# After addition of L2 regularization:
#    Final Validation accuracy: 84.1%
#    Final Test accuracy: 90.6%

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1846.829224
Minibatch accuracy: 12.5%
Validation accuracy: 39.6%
Minibatch loss at step 500: 127.121094
Minibatch accuracy: 88.3%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 10.990211
Minibatch accuracy: 84.4%
Validation accuracy: 84.6%
Minibatch loss at step 1500: 1.481017
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Minibatch loss at step 2000: 0.580636
Minibatch accuracy: 89.8%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 0.704022
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 0.503715
Minibatch accuracy: 92.2%
Validation accuracy: 84.3%
Test accuracy: 90.7%


In [39]:
# Problem 2

batch_size = 128
hidden_nodes = 1024

mini_dataset = batch_size * 3 # This will be used to reduce the size of our dataset to only 3 batches

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  # The shape of weights_in must be the same as the number of columns of the training set times 
  # the number of hidden nodes (each node is a ReLU that will predict one specific thing)
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
  # The shape of weights_out must be the same as the number of columns of weights_in times 
  # the number of labels to finally be output
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation. This was made into a function because this series of operations is performed
  # multiple times in addition to trianing when calculating predictions for the validation and test 
  # sets
  def training_comps(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes layer_1 to be based on ReLU's
      out_layer = tf.matmul(in_layer, out_weights) + out_biases
      return out_layer
  # This beta is the hyperparameter for the L2 regularization
  beta = 0.005
  logits = training_comps(tf_train_dataset, weights_in, biases_in, weights_out, biases_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    (beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits) # Running the softmax on the logits completes the network
  # Run the training computations for the validation and test sets
  valid_prediction = tf.nn.softmax(training_comps(tf_valid_dataset, weights_in, biases_in, weights_out, biases_out))
  test_prediction = tf.nn.softmax(training_comps(tf_test_dataset, weights_in, biases_in, weights_out, biases_out))

# Network Evaluation

# Before reducing batch size:
#    Final Validation accuracy: 84.1%
#    Final Test accuracy: 90.6%
# After reducing batch size:
#    Final Validation accuracy: 76.0%
#    Final Test accuracy: 83.5%


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  train_dataset_chopped = train_dataset[:mini_dataset]
  train_labels_chopped = train_labels[:mini_dataset]
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_chopped.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels_chopped[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Decreasing the number of batches (essentially, restricting the training data) makes the model overfit.
# We can see that the Minibatch accuracy quickly becomes 100%, but the validation and test accuracies 
# are not even close to 100%, which is a clear sign of overfitting.

Initialized
Minibatch loss at step 0: 1859.809814
Minibatch accuracy: 10.9%
Validation accuracy: 28.0%
Minibatch loss at step 500: 128.614319
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 1000: 10.564438
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 1500: 0.997737
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 2000: 0.211571
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 2500: 0.141069
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 3000: 0.131889
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Test accuracy: 83.4%


In [61]:
# Problem 3

# The below code creates a 2-layer neural network from the logistic classifier above. It then evaluates 
# the network on the training, validation and test sets.

# This code is copied from Assignment 2 aside from the L2 regularization added to the loss function.

batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  # The shape of weights_in must be the same as the number of columns of the training set times 
  # the number of hidden nodes (each node is a ReLU that will predict one specific thing)
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
  # The shape of weights_out must be the same as the number of columns of weights_in times 
  # the number of labels to finally be output
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation. This is used only during the training phase.
  def training_comps_dropout(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes in_layer to be based on ReLU's
      drop_layer = tf.nn.dropout(in_layer, 0.3) # This adds dropout to in_layer with 0.5 of the perceptrons being dropped
      out_layer = tf.matmul(drop_layer, out_weights) + out_biases
      return out_layer

  # Training computation. This is used during the validation and testing phases
  def training_comps(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes layer_1 to be based on ReLU's
      out_layer = tf.matmul(in_layer, out_weights) + out_biases
      return out_layer

  # This beta is the hyperparameter for the L2 regularization
  beta = 0.005
  logits = training_comps_dropout(tf_train_dataset, weights_in, biases_in, weights_out, biases_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    (beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits) # Running the softmax on the logits completes the network
  # Run the training computations for the validation and test sets
  valid_prediction = tf.nn.softmax(training_comps(tf_valid_dataset, weights_in, biases_in, weights_out, biases_out))
  test_prediction = tf.nn.softmax(training_comps(tf_test_dataset, weights_in, biases_in, weights_out, biases_out))

# Network Evaluation

# Before dropout:
#    Final Validation accuracy = 81.5%
#    Final Test accuracy = 88.4%
# After dropout:
#    Final Validation accuracy: 83.1%
#    Final Test accuracy: 89.9%

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Adding dropout doesn't seem too have a significant effect on the accuracies of the model, probably
# because this isn't overfitting much because L2 regularization doesn't allow the weights matrix to 
# be changed by much. In fact, dropout seems to actually reduce accuracy slightly, which implies it 
# isn't as helpful for this problem and this dataset.

Initialized
Minibatch loss at step 0: 2211.680176
Minibatch accuracy: 11.7%
Validation accuracy: 30.7%
Minibatch loss at step 500: 135.499603
Minibatch accuracy: 74.2%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 11.425465
Minibatch accuracy: 78.9%
Validation accuracy: 83.1%
Minibatch loss at step 1500: 1.685675
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.678594
Minibatch accuracy: 88.3%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 0.943863
Minibatch accuracy: 78.9%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.803627
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Test accuracy: 89.9%


In [41]:
# Problem 3 (continued)
# 2 layer neural network with overfitting and dropout

batch_size = 128
hidden_nodes = 1024
mini_dataset = batch_size * 3 # This is the size we'll restrict the training set to so that we can overfit

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  # The shape of weights_in must be the same as the number of columns of the training set times 
  # the number of hidden nodes (each node is a ReLU that will predict one specific thing)
  weights_in = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_in = tf.Variable(tf.zeros([hidden_nodes]))
  # The shape of weights_out must be the same as the number of columns of weights_in times 
  # the number of labels to finally be output
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation. This is used only during the training phase.
  def training_comps_dropout(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes in_layer to be based on ReLU's
      drop_layer = tf.nn.dropout(in_layer, 0.3) # This adds dropout to in_layer with 0.5 of the perceptrons being dropped
      out_layer = tf.matmul(drop_layer, out_weights) + out_biases
      return out_layer

  # Training computation. This is used during the validation and testing phases
  def training_comps(in_dataset, in_weights, in_biases, out_weights, out_biases):
      in_layer = tf.matmul(in_dataset, in_weights) + in_biases
      in_layer = tf.nn.relu(in_layer) # This changes layer_1 to be based on ReLU's
      out_layer = tf.matmul(in_layer, out_weights) + out_biases
      return out_layer

  # This beta is the hyperparameter for the L2 regularization
  beta = 0.005
  logits = training_comps_dropout(tf_train_dataset, weights_in, biases_in, weights_out, biases_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    (beta * (tf.nn.l2_loss(weights_in) + tf.nn.l2_loss(weights_out))))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits) # Running the softmax on the logits completes the network
  # Run the training computations for the validation and test sets
  valid_prediction = tf.nn.softmax(training_comps(tf_valid_dataset, weights_in, biases_in, weights_out, biases_out))
  test_prediction = tf.nn.softmax(training_comps(tf_test_dataset, weights_in, biases_in, weights_out, biases_out))

# Network Evaluation

# Before reducing batch size:
#    Final Validation accuracy: 76.0%
#    Final Test accuracy: 84.1%
# After reducing batch size:
#    Final Validation accuracy: 76.3%
#    Final Test accuracy: 83.8%

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  train_data_chopped = train_dataset[:mini_dataset]
  train_labels_chopped = train_labels[:mini_dataset]
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_chopped.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_data_chopped[offset:(offset + batch_size), :]
    batch_labels = train_labels_chopped[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Adding dropout prevents the Minibatch accuracy from immediately skyrocketing to 100%, which helps prevent 
# overfitting


Initialized
Minibatch loss at step 0: 2203.057129
Minibatch accuracy: 9.4%
Validation accuracy: 31.8%
Minibatch loss at step 500: 130.803909
Minibatch accuracy: 98.4%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 10.740642
Minibatch accuracy: 99.2%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 1.036060
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.235738
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 0.162611
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 3000: 0.151951
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Test accuracy: 83.9%


In [50]:
# Problem 4

# The below code takes the network from Problem 3 and attempts to increase its efficiency.
# Features:
# Regularization: L2 and dropout
# 4 hidden layers with each halving in number of nodes
# Exponential learning rate decay

batch_size = 128
hidden_nodes_1 = 1024 #This is the number of hidden nodes in the first layer
hidden_nodes_2 = 512 #This is the number of hidden nodes in the second layer
hidden_nodes_3 = 256 #This is the number of hidden nodes in the third layer
hidden_nodes_4 = 128 #This is the number of hidden nodes in the fourth layer

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  # The shape of weights_1 must be the same as the number of columns of the training set times 
  # the number of hidden nodes (each node is a ReLU that will predict one specific thing)
  # Subsequently, the shape of weights_2 and the following layers must be such that data can be 
  # passed from layer to layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_1]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, hidden_nodes_2]))
  biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))  
  weights_3 = tf.Variable(
    tf.truncated_normal([hidden_nodes_2, hidden_nodes_3]))
  biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
  weights_4 = tf.Variable(
    tf.truncated_normal([hidden_nodes_3, hidden_nodes_4]))
  biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))

  # The shape of weights_out must be the same as the number of columns of the final layer times 
  # the number of labels to finally be output
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_nodes_1, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation. This is used only during the training phase.
  # Arguments:
  # in_dataset: the input dataset
  # weights_list: a list of all the weights variables used for the network
  # biases_list: a list of all the biases used for the network. Must be the same size as weights_list
  # Returns:
  # the result of the network computations at each layer
  def training_comps_dropout(in_dataset, weights_list, biases_list):
      curr_layer = None
      for i in range(0, len(weights_list)):
        if(curr_layer == None): #also happens when i == 0
            curr_layer = tf.matmul(in_dataset, weights_list[0]) + biases_list[0]
        else:
            curr_layer = tf.matmul(curr_layer, weights_list[i]) + biases_list[i]
        if(i != len(weights_list) - 1):
            curr_layer = tf.nn.relu(curr_layer)
            curr_layer = tf.nn.dropout(curr_layer, 0.4)
      return curr_layer

  # Training computation. This is used during the validation and testing phases
  # Arguments:
  # in_dataset: the input dataset
  # weights_list: a list of all the weights variables used for the network
  # biases_list: a list of all the biases used for the network. Must be the same size as weights_list
  # Returns:
  # the result of the network computations at each layer
  def training_comps(in_dataset, weights_list, biases_list):
      curr_layer = None
      for i in range(0, len(weights_list)):
        if(curr_layer == None): #also happens when i == 0
            curr_layer = tf.matmul(in_dataset, weights_list[0]) + biases_list[0]
        else:
            curr_layer = tf.matmul(curr_layer, weights_list[i]) + biases_list[i]
        if(i != len(weights_list) - 1):
            curr_layer = tf.nn.relu(curr_layer)
      return curr_layer

  # This beta is the hyperparameter for the L2 regularization
  beta = 0.005
  list_of_weights = [weights_1, weights_out]
  list_of_biases = [biases_1, biases_out]
  logits = training_comps_dropout(tf_train_dataset, list_of_weights, list_of_biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    (beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_out))))
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 2000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits) # Running the softmax on the logits completes the network
  # Run the training computations for the validation and test sets
  valid_prediction = tf.nn.softmax(training_comps(tf_valid_dataset, list_of_weights, list_of_biases))
  test_prediction = tf.nn.softmax(training_comps(tf_test_dataset, list_of_weights, list_of_biases))

# Network Evaluation

# Before optimization (mainly deepening network):
#    Final Validation accuracy = 83.1%
#    Final Test accuracy = 89.9%
# After optimization:
#    Final Validation accuracy: 83.9%
#    Final Test accuracy: 90.3%

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2076.881348
Minibatch accuracy: 10.9%
Validation accuracy: 32.7%
Minibatch loss at step 500: 130.146759
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 11.208393
Minibatch accuracy: 76.6%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 1.598780
Minibatch accuracy: 80.5%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 0.632579
Minibatch accuracy: 89.1%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.852194
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.713255
Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at step 3500: 0.935209
Minibatch accuracy: 77.3%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 0.914629
Minibatch accuracy: 78.9%
Validation accuracy: 84.2%
Minibatch loss at step 4500: 0.866805
Minibatch accuracy: 79.7%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 0.943296
Minibatch accuracy: 78.9%
Validation a